In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 加载数据
data = pd.read_csv("EI-hp.csv")  # 数据保存在一个 EI-hp.CSV文件中

# 数据预处理
# 选择特征列和目标列
features = data.columns[3:-1]  # 排除前3列和最后1列
target = "AHP"

X = data[features].values
y = data[target].values

# 数据标准化
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# 数据集划分，先划分为训练集和测试集，再从训练集中划分验证集
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 验证集占训练集的25%

# 转换为 PyTorch 张量
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 定义神经网络模型
class HousePricePredictor(nn.Module):
    def __init__(self):
        super(HousePricePredictor, self).__init__()
        self.fc1 = nn.Linear(len(features), 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = HousePricePredictor()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))

    # 反向传播与优化
    loss.backward()
    optimizer.step()

    # 每10个epochs输出一次损失和验证损失
    if (epoch + 1) % 10 == 0:
        model.eval()  # 切换到评估模式
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = mean_squared_error(y_val, val_outputs.view(-1))  # 计算验证集损失
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    y_pred = model(X_test).view(-1)
    test_loss = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test.numpy(), y_pred.numpy())  # 计算R^2
    print(f'Test MSE Loss: {test_loss:.4f}')
    print(f'Test R^2: {r2:.4f}')  # 打印R^2值

# 反归一化预测值和真实值，输出最终的MSE误差
y_pred_original = scaler_y.inverse_transform(y_pred.numpy().reshape(-1, 1))
y_test_original = scaler_y.inverse_transform(y_test.numpy().reshape(-1, 1))
final_mse = mean_squared_error(y_test_original, y_pred_original)
final_r2 = r2_score(y_test_original, y_pred_original)  # 计算反归一化后的R^2
print(f'Final Test MSE (original scale): {final_mse:.4f}')
print(f'Final Test R^2 (original scale): {final_r2:.4f}')  # 打印反归一化后的R^2值


Epoch [10/250], Loss: 0.7572, Validation Loss: 0.9247
Epoch [20/250], Loss: 0.4286, Validation Loss: 0.5626
Epoch [30/250], Loss: 0.1953, Validation Loss: 0.2306
Epoch [40/250], Loss: 0.0801, Validation Loss: 0.0799
Epoch [50/250], Loss: 0.0508, Validation Loss: 0.0689
Epoch [60/250], Loss: 0.0365, Validation Loss: 0.0599
Epoch [70/250], Loss: 0.0269, Validation Loss: 0.0408
Epoch [80/250], Loss: 0.0219, Validation Loss: 0.0391
Epoch [90/250], Loss: 0.0193, Validation Loss: 0.0372
Epoch [100/250], Loss: 0.0168, Validation Loss: 0.0354
Epoch [110/250], Loss: 0.0148, Validation Loss: 0.0322
Epoch [120/250], Loss: 0.0130, Validation Loss: 0.0307
Epoch [130/250], Loss: 0.0114, Validation Loss: 0.0292
Epoch [140/250], Loss: 0.0100, Validation Loss: 0.0273
Epoch [150/250], Loss: 0.0087, Validation Loss: 0.0254
Epoch [160/250], Loss: 0.0075, Validation Loss: 0.0236
Epoch [170/250], Loss: 0.0065, Validation Loss: 0.0220
Epoch [180/250], Loss: 0.0055, Validation Loss: 0.0205
Epoch [190/250], Lo